In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString, MultiLineString
import re
import geopandas as gpd
from shapely.geometry import Point
from shapely.strtree import STRtree
from collections import defaultdict
import geopandas as gpd
from shapely import shortest_line
# Load your lines


wärme  = gpd.read_file(r"C:\Users\rigli\Documents\lk_waerme.gpkg")


In [2]:
output = gpd.GeoDataFrame(columns=["GWR_EGID", "BFSNR", "ARTZH", "Projektiert", "ADRESSE", "PLZ", "ORTSCHAFTSNAME", "tid", "eigentuemer", "VARIANTE", "ID","geometry"])

In [3]:
wärme = wärme.drop(columns=['fileid', 'obj_id','medium', 'datenherr','datenlieferant','letzte_aenderung','lagebestimmung','breite','objektart','letzte_lieferung'])

In [4]:
adressen = gpd.read_file("C:\dev\ews\Adressen_GIS-ZH_-OGD/Adressen_GIS-ZH_-OGD.gpkg")

gebäude = gpd.read_file("C:\dev\ews\AV_MOpublic-_Bodenbedeckung_-OGD\AV_MOpublic-_Bodenbedeckung_-OGD.gpkg", layer="AVZH_BODENBEDECKUNG_F")

proj_gebäude = gpd.read_file("C:\dev\ews\AV_MOpublic-_Bodenbedeckung_-OGD\AV_MOpublic-_Bodenbedeckung_-OGD.gpkg", layer="AVZH_BODENBEDECKUNG_PROJ_F")

liegeschaften = gpd.read_file("C:\dev\ews\AV_MOpublic-_Liegenschaften_-OGD\AV_MOpublic-_Liegenschaften_-OGD.gpkg", layer="AVZH_LIEGENSCHAFTEN_F")

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
C:\Users\rigli\AppData\Local\Temp\ipykernel_11584\525989751.py:1: SyntaxWarning: invalid escape sequence '\d'
  adressen = gpd.read_file("C:\dev\ews\Adressen_GIS-ZH_-OGD/Adressen_GIS-ZH_-OGD.gpkg")
C:\Users\rigli\AppData\Local\Temp\ipykernel_11584\525989751.py:3: SyntaxWarning: invalid escape sequence '\d'
  gebäude = gpd.read_file("C:\dev\ews\AV_MOpublic-_Bodenbedeckung_-OGD\AV_MOpublic-_Bodenbedeckung_-OGD.gpkg", layer="AVZH_BODENBEDECKUNG_F")
C:\Users\rigli\AppData\Local\Temp\ipykernel_11584\525989751.py:5: SyntaxWarning: invalid escape sequence '\d'
  proj_gebäude = gpd.read_file("C:\dev\e

In [5]:
#remove useless attributes
liegeschaften.drop(columns=['OBJID','NBIDENT','NUMMER','VOLLSTAENDIGKEIT','FLAECHENMASS','BFSNR','BEARBEITUNGSDATUM','SHAPE.AREA','SHAPE.LEN'], inplace=True)


#filter only gebäude, remove useless attributes
gebäude = gebäude[(gebäude['ART'] == 'Gebäude')]
proj_gebäude = proj_gebäude[proj_gebäude['ARTZHID'].isin([0, 1, 2, 3, 4, 5, 6, 7])]

gebäude.drop(columns=['OBJID','QUALITAET','ART','BEARBEITUNGSDATUM','ARTCHID','ARTZHID','GVZNUMMER','SHAPE.AREA','SHAPE.LEN'], inplace=True)
proj_gebäude.drop(columns=['OBJID','QUALITAET','ART','BEARBEITUNGSDATUM','ARTCHID','ARTZHID','GVZNUMMER','SHAPE.AREA','SHAPE.LEN','BEWILLIGUNGSSTATUS'], inplace=True)


In [6]:
gebäude['Projektiert'] = 'Nein'

In [7]:
proj_gebäude['Projektiert'] = 'Ja'

In [8]:
all_gebäude = pd.concat([gebäude, proj_gebäude])

all_gebäude =gpd.GeoDataFrame(all_gebäude, geometry='geometry')

In [9]:

# Create GeoDataFrame for first vertices
first_vertices = gpd.GeoDataFrame(
    wärme.drop(columns='geometry'),  # keep other columns if needed
    geometry=wärme.geometry.apply(lambda line: Point(line.coords[0])),
    crs=wärme.crs
)

# Create GeoDataFrame for last vertices
last_vertices = gpd.GeoDataFrame(
    wärme.drop(columns='geometry'),  # keep other columns if needed
    geometry=wärme.geometry.apply(lambda line: Point(line.coords[-1])),
    crs=wärme.crs
)

# Combine into one GeoDataFrame if desired
all_vertices = gpd.GeoDataFrame(
    pd.concat([first_vertices, last_vertices], ignore_index=True),
    crs=wärme.crs
)



In [10]:
# Drop duplicate geometries
unique_vertices = all_vertices.drop_duplicates(subset='geometry')

# Display result
print(unique_vertices)

                    tid           eigentuemer      status  \
0              9E9ABEA6  Stadtwerk Winterthur  in_Betrieb   
1              E99D8E30  Stadtwerk Winterthur  in_Betrieb   
2              7094DF8A  Stadtwerk Winterthur  in_Betrieb   
3              0793EF1C  Stadtwerk Winterthur  in_Betrieb   
4              421FC40E  Stadtwerk Winterthur  in_Betrieb   
...                 ...                   ...         ...   
93309  ch25y5vd0000009q        Gemeinde Bachs  in_Betrieb   
93310  ch25y5vd0000009u        Gemeinde Bachs  in_Betrieb   
93313  ch25y5vd000000a0        Gemeinde Bachs  in_Betrieb   
93316  ch25y5vd000000a6        Gemeinde Bachs  in_Betrieb   
93318  ch25y5vd000000aa        Gemeinde Bachs  in_Betrieb   

                              geometry  
0      POINT (2697364.578 1262468.122)  
1      POINT (2697364.219 1262468.047)  
2      POINT (2697364.379 1262467.659)  
3      POINT (2697373.366 1262481.741)  
4      POINT (2698083.449 1261529.774)  
...                  

In [11]:
points = unique_vertices

# Buffer points by 5 cm (0.05 meters)
points['buffered'] = points.geometry.buffer(0.05)

# Create a GeoDataFrame from buffered geometries for spatial join
buffer_gdf = gpd.GeoDataFrame(points.drop(columns='geometry'), geometry=points['buffered'], crs=points.crs)

# Spatial join: find which buffered points intersect which lines
join = gpd.sjoin(buffer_gdf, wärme, how='left', predicate='intersects')

# Count number of unique lines each buffered point intersects
counts = join.groupby(join.index).size()

# Filter points that intersect less than 2 lines (keep those)
keep_points = counts[counts < 2].index

# Keep original points corresponding to these indices
anschlüsse = points.loc[keep_points]

print(f"Original points count: {len(points)}")
print(f"Filtered points count (intersect < 2 lines): {len(anschlüsse)}")

c:\dev\ews\.venv\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Original points count: 54708
Filtered points count (intersect < 2 lines): 23327


In [12]:
# Drop the extra geometry column ('buffered') before saving
anschlüsse = anschlüsse.drop(columns='buffered', errors='ignore')
# Make sure the active geometry is 'geometry' column
anschlüsse.set_geometry('geometry', inplace=True)
anschlüsse['ID'] = range(len(anschlüsse))
anschlüsse.to_file("wärme_output.gpkg", layer='WÄRMEANSCHLUSS_P')

In [13]:
adressen['ADRESSE'] = adressen['STRASSENNAME'] + ' ' + adressen['HAUSNUMMER']
adressen_clean = adressen[adressen['GWR_EGID'].notna()]
adressen_subset = adressen_clean[['ADRESSE','GWR_EGID','PLZ','ORTSCHAFTSNAME']]

In [15]:
gebäude_adressen = all_gebäude.merge(adressen_subset, how='left', on='GWR_EGID')

In [16]:
def filter_out_number_dot_number(address):
    # Ensure it's a string before applying regex
    address = str(address)
    return not re.search(r'(?:\d+)?\.\d', address)
    
    
valid_gebäude = gebäude_adressen[gebäude_adressen['ADRESSE'].apply(filter_out_number_dot_number)]


valid_gebäude.to_file("wärme_output.gpkg", layer='ALLE_VALIDE_GEBÄUDE_F')
liegeschaften.to_file("wärme_output.gpkg", layer='LIEGENSCHAFTEN_F')

In [17]:
valid_gebäude

,GWR_EGID,BFSNR,ARTZH,geometry,Projektiert,ADRESSE,PLZ,ORTSCHAFTSNAME
0,33318.0,91,Gebäude Landwirtschaft,"POLYGON ((2669654.39 1261177.714, 2669658.36 1...",Nein,Surbengraben 1,8166.0,Niederweningen
4,33109.0,91,Gebäude Wohnen,"POLYGON ((2670096.231 1261845.328, 2670093.111...",Nein,Breitstrasse 1,8166.0,Niederweningen
5,33088.0,91,Gebäude Wohnen,"POLYGON ((2669928.461 1262382.101, 2669936.233...",Nein,Märzenbrunnen 1,8166.0,Niederweningen
6,210195948.0,91,Gebäude Wohnen,"POLYGON ((2669986.357 1262408.551, 2669986.155...",Nein,Haselstrasse 9,8166.0,Niederweningen
7,210215140.0,91,Gebäude Wohnen,"POLYGON ((2670037.308 1262458.57, 2670042.844 ...",Nein,Murzlenstrasse 62,8166.0,Niederweningen
...,...,...,...,...,...,...,...,...
359199,NaN,120,Nebengebäude,"POLYGON ((2711165.654 1237719.097, 2711163.839...",Ja,NaN,NaN,NaN
359200,192058293.0,120,Gebäude Wohnen,"POLYGON ((2711319.308 1236775.877, 2711321.022...",Ja,Birkenstrasse 4,8636.0,Wald ZH
359201,192026188.0,114,Gebäude Wohnen,"POLYGON ((2711377.1 1241441.993, 2711376.704 1...",Ja,Rosenbergstrasse 110b,8498.0,Gibswil
359203,NaN,120,Gebäude Wohnen,"POLYGON ((2712245.232 1240454.44, 2712237.768 ...",Ja,NaN,NaN,NaN


In [ ]:
#spatial join with gebäude
gebäude_anschluss = valid_gebäude.sjoin(anschlüsse, how='inner', predicate='intersects')

#define variante
gebäude_anschluss['VARIANTE'] = 'Anschluss in Gebäude'

output = gpd.GeoDataFrame(pd.concat([output, gebäude_anschluss[["GWR_EGID", "BFSNR", "ARTZH", "Projektiert", "ADRESSE", "PLZ", "ORTSCHAFTSNAME", "tid", "eigentuemer", "VARIANTE", "geometry","ID"]]], ignore_index=True))


C:\Users\rigli\AppData\Local\Temp\ipykernel_3532\630904738.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output = gpd.GeoDataFrame(pd.concat([output, gebäude_anschluss[["GWR_EGID", "BFSNR", "ARTZH", "Projektiert", "ADRESSE", "PLZ", "ORTSCHAFTSNAME", "tid", "eigentuemer", "VARIANTE", "geometry","ID"]]], ignore_index=True))


In [48]:
gebäude_anschluss.to_file("wärme_output.gpkg", layer='ANSCHLUSS_IN_GEBÄUDE_F')

In [49]:
# not_geb_joined = anschlüsse[~anschlüsse['ID'].isin(output['ID'])]

In [50]:

# ─────────────────────────────────────────────────────────────
# Step 1: Spatial join to get parcel ID for points and buildings

anschluss_liegenschaft = gpd.sjoin(
    anschlüsse, liegeschaften, how="left", predicate='within'
)

# Save original geometry and use centroids for spatial join
valid_gebäude['orig_geom'] = valid_gebäude.geometry
valid_gebäude['geometry'] = valid_gebäude['orig_geom'].centroid

gebäude_liegenschaft = gpd.sjoin(
    valid_gebäude, liegeschaften, how="left", predicate='within'
)

# Restore original geometry
gebäude_liegenschaft['geometry'] = gebäude_liegenschaft['orig_geom']
gebäude_liegenschaft = gebäude_liegenschaft.drop(columns=['orig_geom'])


c:\dev\ews\.venv\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [51]:


# Suppose ansch_gdf is your anschluss_liegenschaft GeoDataFrame
# and buildings_gdf is gebäude_liegenschaft

# Optionally, filter only matching parcels first (to reduce candidates)
# For example, do a spatial join or filter buildings by parcel.

def find_best_building_with_line(point_geom, parcel_id, buildings_gdf):
    # Filter buildings in the same parcel (or pre-indexed)
    candidates = buildings_gdf[buildings_gdf['EGRIS_EGRID'] == parcel_id]
    if candidates.empty:
        return None, None  # no match

    best_dist = float('inf')
    best_building = None
    best_line = None

    for _, b in candidates.iterrows():
        b_geom = b.geometry
        # compute shortest line between point and building
        line = shortest_line(point_geom, b_geom)
        d = line.length
        if d < best_dist:
            best_dist = d
            best_line = line
            best_building = b

    return best_building, best_line

# Now apply in a loop (or vectorized) to your anschluss_gdf
results = []
for _, a in anschluss_liegenschaft.iterrows():
    p = a.geometry
    pid = a['EGRIS_EGRID']
    b_row, line = find_best_building_with_line(p, pid, gebäude_liegenschaft)
    results.append({
        'ansch_id': a.name,
        'tid': a['tid'],
        'eigentuemer': a['eigentuemer'],
        'tid': a['status'],
        'ID': a['ID'],
        'GWR_EGID': (b_row['GWR_EGID'] if b_row is not None else None),
        'line': line,
        'distance': line.length if line is not None else None,
        # add other building attributes as needed
    })

# Convert results to a GeoDataFrame if you like
nearest_building_line = gpd.GeoDataFrame(results, geometry='line', crs=gebäude_liegenschaft.crs)


In [52]:
nearest_building_line = nearest_building_line.dropna(subset=['GWR_EGID'])


nearest_building_line.to_file("wärme_output.gpkg", layer='NÄCHSTES_GEBÄUDE_IN_LIEGENSCHAFT_L')

In [ ]:

nearest_building_line_subset = nearest_building_line.drop(columns=['line','ansch_id'])

nearest_building = valid_gebäude.merge(nearest_building_line_subset, on='GWR_EGID')

nearest_building['VARIANTE'] = 'Nächstes Gebäude in Liegenschaft'

output = gpd.GeoDataFrame(pd.concat([output, nearest_building[["GWR_EGID", "BFSNR", "ARTZH", "Projektiert", "ADRESSE", "PLZ", "ORTSCHAFTSNAME", "tid", "eigentuemer", "VARIANTE", "geometry","ID"]]], ignore_index=True))


nearest_building.to_file("wärme_output.gpkg", layer='NÄCHSTES_GEBÄUDE_IN_LIEGENSCHAFT_F')

In [54]:
all_building_in_liegenschaft = anschluss_liegenschaft.merge(gebäude_liegenschaft, on='EGRIS_EGRID')

In [55]:
all_building_in_liegenschaft = all_building_in_liegenschaft.drop(columns=['geometry_x', 'index_right_x', 'index_right_y', 'EGRIS_EGRID'])

In [56]:
# Set the geometry column
all_building_in_liegenschaft = all_building_in_liegenschaft.set_geometry('geometry_y')

# Rename it to 'geometry' if it's not already called that
all_building_in_liegenschaft = all_building_in_liegenschaft.rename_geometry('geometry')

In [57]:
all_building_in_liegenschaft.to_file("wärme_output.gpkg",layer="ALLE_GEBÄUDE_IN_LIEGENSCHAFT_F")

all_building_in_liegenschaft['VARIANTE'] = 'Alle Gebäude in der Liegenschaft'

output = gpd.GeoDataFrame(pd.concat([output, all_building_in_liegenschaft[["GWR_EGID", "BFSNR", "ARTZH", "Projektiert", "ADRESSE", "PLZ", "ORTSCHAFTSNAME", "tid", "eigentuemer", "VARIANTE", "geometry","ID"]]], ignore_index=True))

In [58]:

building_sindex = gebäude_liegenschaft.sindex

def find_best_building_with_line(point_geom, buildings_gdf, sindex, search_radius=100):
    """
    Finds the closest building to a point using the shortest_line.
    Uses spatial index for fast lookup.
    
    Returns:
        - best matching building row (GeoSeries)
        - shortest line (LineString)
    """
    # Bounding box around the point
    bbox = point_geom.buffer(search_radius).bounds
    candidate_idx = list(sindex.intersection(bbox))

    if not candidate_idx:
        return None, None

    candidates = buildings_gdf.iloc[candidate_idx]

    best_dist = float('inf')
    best_building = None
    best_line = None

    for _, b in candidates.iterrows():
        b_geom = b.geometry
        line = shortest_line(point_geom, b_geom)
        d = line.length

        if d < best_dist:
            best_dist = d
            best_line = line
            best_building = b

    return best_building, best_line

# ------------------------------
# Apply to all points in anschluss_liegenschaft
# ------------------------------

results = []

for _, a in anschluss_liegenschaft.iterrows():
    p = a.geometry
    b_row, line = find_best_building_with_line(
        point_geom=p,
        buildings_gdf=gebäude_liegenschaft,
        sindex=building_sindex,
        search_radius=100  # adjust as needed
    )

    results.append({
        'ansch_id': a.name,
        'tid': a.get('tid'),
        'eigentuemer': a.get('eigentuemer'),
        'status': a.get('status'),
        'ID': a.get('ID'),
        'GWR_EGID': b_row.get('GWR_EGID') if b_row is not None else None,
        'line': line,
        'distance': line.length if line is not None else None,
    })

# ------------------------------
# Convert to GeoDataFrame
# ------------------------------

nearest_building_line_all = gpd.GeoDataFrame(results, geometry='line', crs=gebäude_liegenschaft.crs)

In [ ]:
nearest_building_line_all = nearest_building_line_all.dropna(subset=['GWR_EGID'])

nearest_building_line_all.to_file("wärme_output.gpkg", layer='NÄCHSTES_GEBÄUDE_L')

nearest_building_line_all_subset = nearest_building_line_all.drop(columns=['line','ansch_id'])

nearest_building_all = valid_gebäude.merge(nearest_building_line_all_subset, on='GWR_EGID')

nearest_building_all['VARIANTE'] = 'Nächstes Gebäude'

output = gpd.GeoDataFrame(pd.concat([output, nearest_building_all[["GWR_EGID", "BFSNR", "ARTZH", "Projektiert", "ADRESSE", "PLZ", "ORTSCHAFTSNAME", "tid", "eigentuemer", "VARIANTE", "geometry","ID","distance"]]], ignore_index=True))


nearest_building_all.to_file("wärme_output.gpkg", layer='NÄCHSTES_GEBÄUDE_F')

In [60]:
import datetime

today = datetime.date.today()


output['GWR_EGID'] = output['GWR_EGID'].fillna(0).astype(int)
output.to_csv(f"wärme_output_{today}.csv")

In [61]:
nearest_building_line_all.to_file("wärme_output.gpkg", layer='NÄCHSTES_GEBÄUDE_L')


In [62]:
output

,GWR_EGID,BFSNR,ARTZH,Projektiert,ADRESSE,PLZ,ORTSCHAFTSNAME,tid,eigentuemer,VARIANTE,ID,geometry,distance
0,210013026,243,Gebäude Verwaltung,Nein,Fondlistrasse 15,8953.0,Dietikon,1_ch192dx1emDA1HTO,Limeco,Anschluss in Gebäude,10568,"POLYGON ((2671597.38 1251148.25, 2671597.077 1...",NaN
1,210013026,243,Gebäude Verwaltung,Nein,Fondlistrasse 15,8953.0,Dietikon,1_ch192dx1Fd9M9x7m,Limeco,Anschluss in Gebäude,9794,"POLYGON ((2671597.38 1251148.25, 2671597.077 1...",NaN
2,210013026,243,Gebäude Verwaltung,Nein,Fondlistrasse 15,8953.0,Dietikon,1_ch192dx10ETtjxub,Limeco,Anschluss in Gebäude,10567,"POLYGON ((2671597.38 1251148.25, 2671597.077 1...",NaN
3,117885,243,Gebäude Industrie,Nein,Fondlistrasse 7,8953.0,Dietikon,1_ch192dx1ExjMr2Ev,Limeco,Anschluss in Gebäude,10575,"POLYGON ((2671679.24 1251146.971, 2671667.408 ...",NaN
4,117885,243,Gebäude Industrie,Nein,Fondlistrasse 7,8953.0,Dietikon,1_ch192dx1lPKlKi9L,Limeco,Anschluss in Gebäude,9798,"POLYGON ((2671679.24 1251146.971, 2671667.408 ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132895,1162199,230,Gebäude Wohnen,Ja,Hörnlistrasse 24,8400.0,Winterthur,C8DF3CAF,Stadtwerk Winterthur,Nächstes Gebäude,8083,"POLYGON ((2698455.511 1261034.484, 2698457.185...",8.040518
132896,192000928,230,Gebäude Industrie,Ja,Rudolf-Diesel-Strasse 11,8404.0,Winterthur,E8165126,Stadtwerk Winterthur,Nächstes Gebäude,1416,"POLYGON ((2699226.786 1261332.862, 2699231.199...",10.273968
132897,192000928,230,Gebäude Industrie,Ja,Rudolf-Diesel-Strasse 11,8404.0,Winterthur,F6871C53,Stadtwerk Winterthur,Nächstes Gebäude,9470,"POLYGON ((2699226.786 1261332.862, 2699231.199...",10.274123
132898,192058831,228,Gebäude Wohnen,Ja,Schürstrasse 10a,8488.0,Turbenthal,ch23bfex0000ww2c,Privat,Nächstes Gebäude,20447,"POLYGON ((2706081.379 1255484.172, 2706069.519...",0.011950
